In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import KFold, train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV

from imblearn.under_sampling import NearMiss
from collections import Counter
import gc

In [1]:
df_reduced_quant = pd.read_csv("df_reduced_quant.csv")

NameError: name 'pd' is not defined

In [15]:
df = df_reduced_quant

In [16]:
df.dtypes.value_counts()

float64    21
int64      18
Name: count, dtype: int64

In [17]:
def reduzir_tipos(df):
    df_otimizado = df.copy()

    for col in df.columns:
        col_data = df[col]

        if pd.api.types.is_numeric_dtype(col_data):
            col_min = col_data.min()
            col_max = col_data.max()

            # transforma em booleano se tiver só 0 e 1
            if set(col_data.unique()).issubset({0, 1}):
                df_otimizado[col] = col_data.astype(bool)
                continue

            # inteirso
            if pd.api.types.is_integer_dtype(col_data):
                if np.iinfo(np.int8).min <= col_min and col_max <= np.iinfo(np.int8).max:
                    df_otimizado[col] = col_data.astype(np.int8)
                elif np.iinfo(np.int16).min <= col_min and col_max <= np.iinfo(np.int16).max:
                    df_otimizado[col] = col_data.astype(np.int16)
                elif np.iinfo(np.int32).min <= col_min and col_max <= np.iinfo(np.int32).max:
                    df_otimizado[col] = col_data.astype(np.int32)
                else:
                    df_otimizado[col] = col_data.astype(np.int64)

            #floats
            elif pd.api.types.is_float_dtype(col_data):
                if np.finfo(np.float16).min <= col_min and col_max <= np.finfo(np.float16).max:
                    df_otimizado[col] = col_data.astype(np.float16)
                elif np.finfo(np.float32).min <= col_min and col_max <= np.finfo(np.float32).max:
                    df_otimizado[col] = col_data.astype(np.float32)
                else:
                    df_otimizado[col] = col_data.astype(np.float64)

    return df_otimizado
df_teste = reduzir_tipos(df)
df_teste.dtypes.value_counts()

int32      15
float32    11
float16     8
float64     2
int8        2
int64       1
Name: count, dtype: int64

In [19]:
df_teste.to_parquet("df_teste.parquet", index=False)

In [4]:
df_teste1 = pd.read_parquet("df.parquet")
df_teste1.dtypes.value_counts()

int32      15
float32    11
float16     8
float64     2
int8        2
int64       1
Name: count, dtype: int64

In [27]:
del  df_teste1
gc.collect()

0

In [6]:
df_teste1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9430663 entries, 0 to 9430662
Data columns (total 39 columns):
 #   Column             Dtype  
---  ------             -----  
 0   fwd_iat_mean       float32
 1   bwd_pkt_len_mean   float16
 2   fwd_iat_tot        float32
 3   pkt_len_var        float32
 4   init_fwd_win_byts  int32  
 5   subflow_bwd_byts   int32  
 6   pkt_len_mean       float16
 7   pkt_size_avg       float16
 8   bwd_header_len     int32  
 9   totlen_bwd_pkts    float32
 10  fwd_act_data_pkts  int32  
 11  flow_byts/s        float64
 12  flow_iat_mean      float32
 13  fwd_pkts/s         float32
 14  tot_bwd_pkts       int32  
 15  flow_pkts/s        float64
 16  bwd_pkt_len_std    float16
 17  pkt_len_std        float16
 18  bwd_seg_size_avg   float16
 19  fwd_iat_min        float32
 20  pkt_len_max        int32  
 21  flow_iat_max       float32
 22  tot_fwd_pkts       int32  
 23  bwd_pkt_len_max    int32  
 24  bwd_pkts/s         float32
 25  flow_iat_min      

In [7]:
x = df_otimizado.drop('label', axis=1)
y = df_otimizado['label']

x_clean = x.replace([np.inf, -np.inf], np.nan)
x_clean = x_clean.fillna(x_clean.mean())

x1 = x_clean.values
y1 = y.values

In [8]:
del df_otimizado, x, y, x_clean

In [9]:
rf = RandomForestClassifier(n_estimators=200, max_depth=30, n_jobs=-1, random_state=42)
kf = KFold(n_splits=10, shuffle=True, random_state=42)

test_scores_RF_mdi, precision_scores_RF_mdi, recall_scores_RF_mdi, f1_scores_RF_mdi = [], [], [], []
feature_importances_list = []

rf_base = RandomForestClassifier(max_features='sqrt', random_state=42)

In [10]:
X_train, X_test, Y_train, Y_test = train_test_split(x1, y1, test_size=0.2, stratify=y1, random_state=42)

rf.fit(X_train, Y_train)

feature_importances_list.append(rf.feature_importances_)

y_pred = rf.predict(X_test)
test_scores_RF_mdi.append(accuracy_score(Y_test, y_pred))
precision_scores_RF_mdi.append(precision_score(Y_test, y_pred, average='weighted'))
recall_scores_RF_mdi.append(recall_score(Y_test, y_pred, average='weighted'))
f1_scores_RF_mdi.append(f1_score(Y_test, y_pred, average='weighted'))

del X_train, X_test, Y_train, Y_test, rf, y_pred
gc.collect()

72

In [13]:
# Redução de features
importances_mean = np.mean(feature_importances_list, axis=0)
importances_series = pd.Series(importances_mean, index=df_otimizado.drop('label', axis=1).columns)
selected_features = importances_series[importances_series >= importances_series.median()].index.tolist()
df_reduced = df_otimizado[selected_features + ['label']]

In [14]:
metrics_mdi = pd.DataFrame({
    "Modelo": ["RandomForest"],
    "Acurácia": [np.mean(test_scores_RF_mdi)],
    "Precisão": [np.mean(precision_scores_RF_mdi)],
    "Recall": [np.mean(recall_scores_RF_mdi)],
    "F1 Score": [np.mean(f1_scores_RF_mdi)]
})

metrics_mdi.to_csv("metrics_mdi.csv", index=False)
df_reduced.to_csv("df_reduced_quant.csv", index=False)

In [17]:
df_reduced.columns.__len__()

39

In [5]:
df_sampled = df.groupby('label').apply(lambda x: x.sample(n=1000, random_state=42)).reset_index(drop=True)

C:\Users\mikae\AppData\Local\Temp\ipykernel_3068\286496714.py:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_sampled = df.groupby('label').apply(lambda x: x.sample(n=1000, random_state=42)).reset_index(drop=True)


In [6]:
df_sampled.to_csv("df_sampled.csv", index=False)

In [7]:
df_sampled['label'].value_counts()

label
0    1000
1    1000
2    1000
3    1000
4    1000
Name: count, dtype: int64

In [7]:
import os
import gc
import joblib
import numpy as np
import pandas as pd

from collections import Counter
from pandas.util import hash_pandas_object

from sklearn.model_selection import KFold, train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

from imblearn.under_sampling import NearMiss
from scipy.stats import wilcoxon

In [9]:
df_reduced = pd.read_parquet("df.parquet")

In [10]:
 # KFold
kf = KFold(n_splits=10, shuffle=True, random_state=42)
scaler = StandardScaler()

x = df_reduced.drop('label', axis=1)
y = df_reduced['label']

x_clean = x.replace([np.inf, -np.inf], np.nan)
x_clean = x_clean.fillna(x_clean.mean())

x1 = x_clean.values
y1 = y.values

c:\Users\mikae\toCode\Artigo_IA\ondocker\venv\Lib\site-packages\pandas\core\nanops.py:1496: RuntimeWarning: overflow encountered in cast
  return count.astype(dtype, copy=False)
c:\Users\mikae\toCode\Artigo_IA\ondocker\venv\Lib\site-packages\numpy\_core\_methods.py:52: RuntimeWarning: overflow encountered in reduce
  return umr_sum(a, axis, dtype, out, keepdims, initial, where)


In [11]:
del df_reduced, x, y, x_clean
gc.collect()

3717

In [ ]:
for fold_index, (train_index, test_index) in enumerate(kf.split(x1)):
    X_train, X_test = x1[train_index], x1[test_index]
    Y_train, Y_test = y1[train_index], y1[test_index]
        
    #isso aqui é o nearmiss e salvando os dados descartados concatenando no conjunto de testes
    counts = Counter(Y_train)
    valid_classes = {cls: 5000 for cls in range(5) if counts.get(cls, 0) >= 5000}

    nm = NearMiss(sampling_strategy=valid_classes, version=2, n_neighbors=3, n_jobs=-1)
    X_resampled, Y_resampled = nm.fit_resample(X_train, Y_train)

    hash_train = hash_pandas_object(pd.DataFrame(X_train)).values
    hash_resampled = hash_pandas_object(pd.DataFrame(X_resampled)).values
    selected_mask = np.isin(hash_train, hash_resampled)

    X_discarded = X_train[~selected_mask]
    Y_discarded = Y_train[~selected_mask]

    X_test = np.concatenate((X_test, X_discarded), axis=0)
    Y_test = np.concatenate((Y_test, Y_discarded), axis=0)

    X_train_val, X_test_val, y_train_val, y_test_val = train_test_split(X_train, Y_train, random_state=42, test_size=0.2)

    #normalizaçao dos dados
    X_train_val = scaler.fit_transform(X_train_val) 
    X_test_val = scaler.transform(X_test_val)
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)
    
    os.makedirs(f"fold_{fold_index}", exist_ok=True)

    pd.DataFrame(X_train_val).to_parquet(f"fold_{fold_index}/X_train_val_fold{fold_index}.parquet", index=False)
    pd.DataFrame(X_test_val).to_parquet(f"fold_{fold_index}/X_test_val_fold{fold_index}.parquet", index=False)
    pd.DataFrame(y_train_val).to_parquet(f"fold_{fold_index}/Y_train_val_fold{fold_index}.parquet", index=False)
    pd.DataFrame(y_test_val).to_parquet(f"fold_{fold_index}/Y_test_val_fold{fold_index}.parquet", index=False)
    pd.DataFrame(X_train).to_parquet(f"fold_{fold_index}/X_train_fold{fold_index}.parquet", index=False)
    pd.DataFrame(X_test).to_parquet(f"fold_{fold_index}/X_test_fold{fold_index}.parquet", index=False)
    pd.DataFrame(Y_train).to_parquet(f"fold_{fold_index}/Y_train_fold{fold_index}.parquet", index=False)
    pd.DataFrame(Y_test).to_parquet(f"fold_{fold_index}/Y_test_fold{fold_index}.parquet", index=False)
    
    del X_train, X_test, Y_train, Y_test, X_train_val, X_test_val, y_train_val, y_test_val, X_resampled, Y_resampled, X_discarded, Y_discarded, hash_train, hash_resampled, selected_mask
    gc.collect()



MemoryError: Unable to allocate 6.44 TiB for an array with shape (6191212, 142914) and data type int64